In [8]:
%run extract_contours

In [4]:
%who

calculate_image_seq_on_dicompyler	 cffas	 contours	 current_file	 current_filename	 current_uid	 display_scan	 file_of_slice_with_contours	 filename	 
find_correct_image	 i	 image_uids	 integervalue	 math	 matplotlib	 np	 pixel_array	 plt	 
pydicom	 uid_to_filename_map	 uid_to_location_map	 x_coords_of_contours	 y_coords_of_contours	 


In [1]:
def find_correct_image(contoursequence):
    """
    Given an rtstruct.dicom files ROIContourSequence[9].ContourSequence[i],
    return the correct filename (using the UID)
    """
    return uid_to_filename_map[contoursequence.ContourImageSequence[0].ReferencedSOPInstanceUID]

In [ ]:
import math
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import matplotlib.path
%matplotlib inline

In [ ]:
def calculate_image_seq_on_dicompyler(image_no):
    return 136-image_no + 1

In [ ]:
def display_scan(filename_text):
    plt.imshow(pydicom.dcmread(filename_text).pixel_array, cmap=plt.cm.bone)

In [ ]:
integervalue = 3
'{0:03d}'.format(integervalue)
# image_uids --> sop_instance_ids_of_136_scans
image_uids = [] # data: ids of all images in the dataset
#image_locations = [] # data : locations of all slices/images in the dataset
uid_to_filename_map = dict()
uid_to_location_map = dict()
filename = 'data/P1152/P1152_pCT_20140226_'

# making a list of UIDs of all images
# going through all the files
for i in range(1, 137):
    current_filename = filename + '{0:03d}'.format(i) + '.dcm'
    current_file = pydicom.dcmread(current_filename)
    current_uid = current_file.SOPInstanceUID
    current_location = current_file.SliceLocation
    image_uids.append(current_uid)
    uid_to_filename_map[current_uid] = current_filename
    uid_to_location_map[current_uid] = current_location # think about merging these 2 dicts
    #image_locations.append(current_file.SliceLocation)  # think about using strings vs integers
    # computational cost.

In [ ]:
def find_correct_image(contoursequence):
    """
    Given an rtstruct.dicom files ROIContourSequence[9].ContourSequence[i],
    return the correct filename (using the UID)
    """
    return uid_to_filename_map[contoursequence.ContourImageSequence[0].ReferencedSOPInstanceUID]

In [ ]:
# contour_file_for_all_136_scans --> cffas
cffas = pydicom.dcmread('data/P1152/P1152_RTStructure_1.dcm')

file_of_slice_with_contours = pydicom.dcmread(find_correct_image(cffas.ROIContourSequence[9].ContourSequence[6]))
pixel_array = file_of_slice_with_contours.pixel_array

contours = cffas.ROIContourSequence[9].ContourSequence[6].ContourData
x_coords_of_contours = [ float(number) for number in contours[0::3] ]
y_coords_of_contours = [ float(number) for number in contours[1::3] ]

In [ ]:
pixel_spacing = file_of_slice_with_contours.PixelSpacing
x_gap = pixel_spacing[0]
y_gap = pixel_spacing[1]
image_position_starting_points = file_of_slice_with_contours.ImagePositionPatient
x_start = float(image_position_starting_points[0])
y_start = float(image_position_starting_points[1])
x_pixel_nos = (np.array(x_coords_of_contours)-x_start)/x_gap
y_pixel_nos = (np.array(y_coords_of_contours)-y_start)/y_gap

In [ ]:
plt.figure()
plt.imshow(pixel_array, cmap='bone')
plt.plot(x_pixel_nos, y_pixel_nos, color='r')
plt.figure()

In [ ]:
# created a list of Nx1x2 numpy arrays
temp_list = []
for a, b in zip(x_pixel_nos, y_pixel_nos):
    temp_list.append([[a, b]])
temp_array = np.array(temp_list)
temp_array_list = []
temp_array_list.append(temp_array)

In [ ]:
temp_array_list[0].shape

In [ ]:
len(contours[0::3])

In [ ]:
#x_pixel_nos = [...]
#y_pixel_nos = [...] # Data from https://gist.github.com/sdoken/173fae1f9d8673ffff5b481b3872a69d

temp_list = []
for a, b in zip(x_pixel_nos, y_pixel_nos):
   temp_list.append([a, b])

polygon = np.array(temp_list)
print(polygon.shape)
np.min(polygon, axis=0) # np.min takes the minimum value in each column (across axis=0 rows), 
# not sure why this is called left and right
left = np.min(polygon, axis=0)   # i think left and right are misnomers here?
right = np.max(polygon, axis=0)  

# i think this just creates a range between the lowest x and highest
x = np.arange(math.ceil(left[0]), math.floor(right[0]) + 1) 
y = np.arange(math.ceil(left[1]), math.floor(right[1]) + 1)

xv, yv = np.meshgrid(x, y, indexing='xy') # do not understand meshgrid...
points = np.hstack((xv.reshape((-1,1)), yv.reshape((-1,1)))) # points created 

path = matplotlib.path.Path(polygon)  # path created from polygon
mask = path.contains_points(points)   # i think what is going on here is that we are getting to see what points 
# are within the path/polygon and what points are not within the 
#mask.shape = xv.shapes

In [ ]:
mesh_matrix = np.meshgrid(x, y, indexing='xy')

In [ ]:
len(mesh_matrix)

In [ ]:
mesh_matrix[0].shape

In [ ]:
help(np.meshgrid)

In [ ]:
temp_list = []
for a, b in zip(x_pixel_nos, y_pixel_nos):
    temp_list.append([a, b])

In [ ]:
?temp_list

In [ ]:
temp_list = []
for a, b in zip(x_pixel_nos, y_pixel_nos):
   temp_list.append([a, b])

polygon = np.array(temp_list)
left = np.min(polygon, axis=0)  # Finds the smallest x coordinate and pairs it with the smallest y-coordinate
right = np.max(polygon, axis=0)
x = np.arange(math.ceil(left[0]), math.floor(right[0])+1)
y = np.arange(math.ceil(left[1]), math.floor(right[1])+1)
xv, yv = np.meshgrid(x, y, indexing='xy') # do not understand meshgrid...
points = np.hstack((xv.reshape((-1,1)), yv.reshape((-1,1)))) # points created 

path = matplotlib.path.Path(polygon)  # path created from polygon
mask = path.contains_points(points)   # i think what is going on here is that we are getting to see what points 
# are within the path/polygon and what points are not within the 
#mask.shape = xv.shapes

In [ ]:
points[mask, :].shape

Plot this this to see whether it includes the points inside.

Check if the points you get from a loop are the same number:

In [ ]:
512*512

In [ ]:
444/(512*512)

In [ ]:
pixel_array.shape

In [ ]:
len(contours)/3

In [ ]:
np.dot(points, mask)

In [ ]:
points.shape

In [ ]:
mask.shape

In [ ]:
For questions, ask bordenmike518@gmail.com

In [ ]:
mask.shape

In [ ]:
points.shape

In [ ]:
points

The points here that are positive/True are the points that fall within the contours.

Okay so this problem has been solved but how??

In [ ]:
temp_list

In [ ]:
type(temp_list)

In [ ]:
len(temp_list)

In [ ]:
type(polygon)

In [ ]:
polygon.shape

In [ ]:
type(mask)

In [ ]:
mask.shape